In [ ]:
# create encoder files for categorical columns and store the files
# status: needs to be cleaned up

In [1]:
import dfBasics
import common
import encoder
import pfAdapt
import charts

Setup Complete


In [2]:
import pandas as pd
from pyspark.sql import functions

In [3]:
sparkSession = dfBasics.getSparkSession()

In [4]:
#df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*').select(columns).dropDuplicates() 
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*').dropDuplicates() 

In [5]:
columns = df.limit(1).toPandas().columns

# Create Encoder

In [6]:
withoutColumns = ['CGLOBALMESSAGEID','CSTARTTIME', 'CENDTIME','CINBOUNDSIZE', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME']
columns = columns[~columns.isin(withoutColumns)]

In [7]:
for column in columns:
    print(column)
    df4 = df.select(column).dropDuplicates().toPandas() 
    df4[column] = df4[column].astype(str)
    encoder.createEncoders(df4,[column])

CSTATUS
['COMPONENT_ERROR' 'COMPONENT_ERROR_CONTENTEXTRACTOR'
 'CONFIGURATION_ERROR' 'DATA_ERROR' 'DELETED' 'ECHO-ERROR' 'ERROR'
 'EXPIRED' 'FAILED' 'MDN_ERROR' 'MESSAGE_QUEUED' 'PENDING'
 'READY_FOR_DOWNLOAD' 'RECALLED' 'SUCCESS' 'SUCCESS_DOWNLOADED'
 'SUCCESS_POLLQUEUE' 'THREAT_DETECTED' 'TORESEND' 'WARNING']
CSERVICE
['COM.Routing.SupplierPortal' 'Content-Based Routing' 'EchoService' 'IDS'
 'InvoicePortal' 'MAKO_BE_ROUTER' 'MaKo' 'Metadata-Based Routing' 'None'
 'PEPPOL' 'SFID Metadata-Based Routing' 'SSP' 'VAN Routing' 'VAT.UK']
CSLABILLINGMONTH
['2019-10' '2019-11' '2019-12' '2020-01' '2020-02' '2020-03' '2020-04'
 '2020-05' '2020-06' '2020-07' '2020-08' '2020-09' '2020-10' '2020-11'
 '2020-12' '2021-01' '2021-02' '2021-03' '2021-04' '2021-05' '2021-06'
 '2021-07' '2021-08' '2021-09' '2021-10' '2021-11' '2021-12' '2022-01'
 '2022-02' '2022-03' '2022-04' '2022-05' '2022-06' '2022-07' '2022-08'
 '2022-09' '2022-10' '2022-11' '2022-12' '2023-01' '2023-02' '2023-03'
 '2023-04' '2023-0

### merge CSENDERENDPOINTID and CRECEIVERENDPOINTID

In [26]:
!mv CSENDERENDPOINTID.npy single_CSENDERENDPOINTID.npy 
!mv CRECEIVERENDPOINTID.npy single_CRECEIVERENDPOINTID.npy

In [12]:
column = 'CSENDERENDPOINTID'
df4_1 = df.select(column).dropDuplicates().toPandas()
column = 'CRECEIVERENDPOINTID'
df4_2 = df.select(column).dropDuplicates().toPandas()

In [24]:
endpoints = list(set(list(df4_1['CSENDERENDPOINTID']) + list(df4_2['CRECEIVERENDPOINTID'])))
df4 = pd.DataFrame(columns=['CSENDERENDPOINTID'])
df4['CSENDERENDPOINTID'] = endpoints 

In [28]:
encoder.createEncoders(df4,['CSENDERENDPOINTID'])

['000b4e02-7f43-11ec-ae40-78d9ac1e124d'
 '0014de30-f8ec-11eb-9933-02daac1e124d'
 '00171070-9207-11eb-9829-4c38ac1e124c' ...
 'fff20390-7cce-11eb-8926-4191ac1e124c'
 'fff8bb00-a213-11e9-a189-ddafac1b495c' None]


In [29]:
!cp CSENDERENDPOINTID.npy CRECEIVERENDPOINTID.npy

### move npy files in versioned folder

In [30]:
!mkdir -p v00003/npy
!mv *.npy v00003/npy

In [31]:
!mv v00003 /home/jovyan/work/share/sla

mv: cannot move 'v00003' to '/home/jovyan/work/share/sla/v00003': Permission denied


# append new encoding classes

In [ ]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00001/v00002/encoded/all/*/*').dropDuplicates() 
columns = df.limit(1).toPandas().columns

In [ ]:
import pyspark.sql.functions as f

In [ ]:
#!ls /tmp

In [ ]:
withoutColumns = ['CGLOBALMESSAGEID','CSTARTTIME', 'CENDTIME','CINBOUNDSIZE', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME','year', 'month', 'day', 'hour', 'minute', 'error']
columns = columns[~columns.isin(withoutColumns)]
columns

In [ ]:
df_org  = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*').dropDuplicates() 

In [ ]:
column = 'CSERVICE'
df4 = df_org.select(column).dropDuplicates().toPandas() 
df4[column] = df4[column].astype(str)
df4 = df4[df4[column] != 'None']


In [ ]:
npy= share_directory + 'sla/' + version_sla + '/npy'
_encoder = TolerantLabelEncoder(ignore_unknown=True)
_encoder.classes_ = np.load(npy + '/' + column + '.npy')

In [ ]:
_encoder.append(_encoder.diff(list(df4[column])))

In [ ]:
new_version_sla = 'v00002'

In [ ]:
new_directory = home_directory + 'output/' + new_version_sla + '/'

In [ ]:
for column in columns:
    print(column)
    df4 = df_org.select(column).dropDuplicates().toPandas() 
    df4[column] = df4[column].astype(str)
    df4 = df4[df4[column] != 'None']
    
    npy= share_directory + 'sla/' + version_sla + '/npy'
    _encoder = TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    
    _encoder.append(_encoder.diff(list(df4[column])))
    np.save(new_directory + column + '.npy', _encoder.classes_)

## test

In [ ]:
#npy= share_directory + 'sla/' + version_sla + '/npy'
column = columns[5]
npy=  new_directory 
_encoder = TolerantLabelEncoder(ignore_unknown=True)
#_encoder.classes_ = np.load(npy + '/' + column + '.npy')
_encoder.classes_ = np.load(npy + column + '.npy')
_encoder.classes_

In [ ]:
columns

In [ ]:
%run encoder.ipynb

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import column_or_1d

class TolerantLabelEncoder(LabelEncoder):
    def __init__(self, ignore_unknown=False,
                       unknown_original_value='unknown', 
                       unknown_encoded_value=-1):
        self.ignore_unknown = ignore_unknown
        self.unknown_original_value = unknown_original_value
        self.unknown_encoded_value = unknown_encoded_value

    def transform(self, y):
        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        indices = np.isin(y, self.classes_)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.searchsorted(self.classes_, y)
        y_transformed[~indices]=self.unknown_encoded_value
        return y_transformed

    # for appended versions
    def transform_version(self, y):
        results = self.transform(y)
        index = np.where(results == -1)[0]
        with_prefix = ['zzzz_0001_' + y[i] for i in index]
        results_with_prefix = self.transform(with_prefix)
        for i,r in enumerate(results_with_prefix):
            results[index[i]] = r
        return results    
    
    def inverse_transform(self, y):
        check_is_fitted(self, 'classes_')

        labels = np.arange(len(self.classes_))
        indices = np.isin(y, labels)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.asarray(self.classes_[y], dtype=object)
        y_transformed[~indices]=self.unknown_original_value
        return y_transformed

    # problem with None
    def dict(self):
        return dict(zip(self.classes_, self.transform(self.classes_)))
    
    def diff(self,values):
        _encoded = list(_encoder.classes_)
        try:
            _encoded.remove(None)
        except Exception as exception:
            pass
        return list(set(values) - set(_encoded))
    
    def append(self,values, version='0001'):
        self.classes_ = np.append(self.classes_, ['zzzz' + '_' + version + '_' + s  for s in list(np.sort(list(values)))])

In [ ]:
import dfBasics
import pandas as pd

version_sla = 'v00001'
version     = version_sla + '/v00000'

home_directory  =  '/home/jovyan/work/'
share_directory =  '/home/jovyan/work/share/'
#share_directory =  '/home/jovyan/share/'

## init encoders (run only once)

In [ ]:
# - run this paragraph only once !
# - if you get an 'allow_pickle' error you need Kernel/restart kernel

import encoder
import numpy as np

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# restore np.load for future normal usage
#np.load = np_load_old

### encode value

In [ ]:
def transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    return int( _encoder.transform([value])[0])

### decode value

In [ ]:
def inverse_transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    if type(value) == int:
        return str(_encoder.inverse_transform(value))  
    elif type(value) == list:
        return [str(_encoder.inverse_transform(v)) for v in value]
    else:
        return None

In [ ]:
inverse_transform([6,5],column='CSERVICE')
inverse_transform(6, 'CSTATUS')
#inverse_transform(list(pd.unique(pfall['CSTATUS'])),'CSTATUS')

# replace -1

In [ ]:
"""
# check for rows with -1
year = 2023
month = 5
df2 = df.where(f.col("year").isin([year])).where(f.col("month").isin([month]))
"""

In [ ]:
"""
df3 = df2.where((f.col('CSTATUS')==-1) | (f.col('CSERVICE')==-1) | (f.col('CSENDERENDPOINTID')==-1) | (f.col('CSENDERPROTOCOL')==-1)| (f.col('CRECEIVERPROTOCOL')==-1) | (f.col('CRECEIVERENDPOINTID')==-1))
ids =  np.array(df3.select('CGLOBALMESSAGEID').drop_duplicates().collect())     
ids = [i[0] for i in ids.tolist()]
not_ids = np.array(df2.select('CGLOBALMESSAGEID').filter(df2.CGLOBALMESSAGEID.isin(ids) == False).drop_duplicates().collect())  
not_ids = [i[0] for i in not_ids.tolist()]
"""

In [ ]:
"""
df_test = df_org.where(f.col("CGLOBALMESSAGEID").isin([ids[0]]))
result = process(dataframe=df_test, encoders=encoders, columns=columns)
result.head()
"""

# work

In [ ]:
_encoder.classes_

In [ ]:
def process(year,month,columns):
    df4 = df3.where(f.col("year").isin([year])).where(f.col("month").isin([month]))
    df4.write.mode('overwrite').parquet('/home/jovyan/work/output/sla_enc_all_v00001_v00002_' + str(year) + '_' + str(month) + '.parquet')

In [ ]:
# check for rows with -1
year = 2023
month = 5
df2 = df.where(f.col("year").isin([year])).where(f.col("month").isin([month]))

In [ ]:
ids = list(df2.where((f.col('CSTATUS')==-1) | (f.col('CSERVICE')==-1) | (f.col('CSENDERENDPOINTID')==-1) | (f.col('CSENDERPROTOCOL')==-1)| (f.col('CRECEIVERPROTOCOL')==-1) | (f.col('CRECEIVERENDPOINTID')==-1)).select('CGLOBALMESSAGEID').toPandas()['CGLOBALMESSAGEID'])

In [ ]:
len(ids)

In [ ]:
columns

In [ ]:
%run functions_encode.ipynb

In [ ]:
dataframe = df2
df3 = dataframe.where(f.col("CGLOBALMESSAGEID").isin([ids[0]]))
df3.show()

In [ ]:
dataframe = df_org
selected_columns = ['CGLOBALMESSAGEID',  'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', 'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME']
#df3 = dataframe.where(f.col("CGLOBALMESSAGEID").isin([ids[0]]))
df3 = dataframe.where(f.col("CGLOBALMESSAGEID") == ids[0]).select(selected_columns).dropDuplicates() 
df3.show()

In [ ]:
test = df_org.where(f.col("CRECEIVERPROTOCOL") == 'N/A')
test.show(1)

In [ ]:
selected_columns

## load encoders

In [ ]:
# load encoders

#npy='/home/jovyan/work/npy'
npy=  new_directory
encoders = {}
for column in columns:
    #_encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder = TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    encoders[column] = _encoder

In [ ]:
encoders

In [ ]:
import numpy as np
import encoder
from pyspark.sql.functions import col
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def transform(value):
    return int( _encoder.transform([value])[0])

column = 'CSENDERPROTOCOL'
_encoder = encoders[column]
dataframe = df3

udf_transform = udf(lambda z: transform(z), StringType())
df4 = dataframe.withColumn(column, udf_transform(col(column)).cast("Integer"))

In [ ]:
df4.show()

# fix replace prefix

In [ ]:
column = 'CRECEIVERENDPOINTID'
_encoder = encoders[column]

In [ ]:
#_encoder.classes_.tolist()
_encoder.classes_

In [ ]:
def fix_prefix(s):
    version = '0001'
    if s.startswith('____0001_'):
        return 'zzzz' + '_' + version + '_' + s.split('____0001_')[1]
    else:
        return s
    
_encoder.classes_ = np.sort(np.array([fix_prefix(s) for s in _encoder.classes_.tolist()]).astype(object))   
encoders[column] = _encoder

new_directory = home_directory + 'output/' + new_version_sla + '/'
np.save(new_directory + column + '.npy', _encoder.classes_)

_encoder.classes_

# fix None Encoder Problem

In [ ]:
encoders

In [ ]:
column = 'CSENDERENDPOINTID'
_encoder = encoders[column]
#np.sort(_encoder.classes_)

In [ ]:
classes = _encoder.classes_
len(classes)
classes

In [ ]:
#check for None
index = np.where(_encoder.classes_ == None)[0][0]
index

In [ ]:
_encoder.inverse_transform([9])

In [ ]:
_encoder.transform(['zzzz_0001_fe99e900-34c3-11ed-91f0-6348ac1e124d_part'])
#_encoder.transform(['____0001_fe99e900-34c3-11ed-91f0-6348ac1e124d_part'])    
_encoder.transform(['00171070-9207-11eb-9829-4c38ac1e124c'])

In [ ]:
_encoder.inverse_transform([6146])

In [ ]:
y=['zzzz_0001_fe99e900-34c3-11ed-91f0-6348ac1e124d_part','fekkkk900-34c3-11ed-91f0-6348ac1e124d_part','fe99e900-34c3-11ed-91f0-6348ac1e124d_part','ffe4d710-d9ec-11ed-9913-0514ac1e124d']
results = np.isin(y, _encoder.classes_)
#np.where(results == False)[0][0]
results

In [ ]:
_encoder.transform_version(['ffe4d710-d9ec-11ed-9913-0514ac1e124d'])
#_encoder.transform_version(['fe99e900-34c3-11ed-91f0-6348ac1e124d_part'])
_encoder.transform_version(y)

In [ ]:
_encoder.transform(['____0001_ffe4d710-d9ec-11ed-9913-0514ac1e124d'])
#_encoder.transform(['____0001_fe99e900-34c3-11ed-91f0-6348ac1e124d_part'])

In [ ]:
results = _encoder.transform(y)
index = np.where(results == -1)[0]
with_prefix = ['____0001_' + y[i] for i in index]
results_with_prefix = _encoder.transform(with_prefix)
for i,r in enumerate(results_with_prefix):
    results[index[i]] = r

In [ ]:
results_with_prefix
with_prefix

In [ ]:
_encoder.transform(with_prefix)

In [ ]:
_encoder.inverse_transform(6148)

In [ ]:
_encoder.transform(['____0001_ffe4d710-d9ec-11ed-9913-0514ac1e124d'])

In [ ]:
_encoder.__dict__

In [ ]:
len(_encoder.classes_)

In [ ]:
index,results

In [ ]:
_encoder.transform_version(y)

In [ ]:
_encoder.transform(y)

In [ ]:
y = ['____0001_ffe4d710-d9ec-11ed-9913-0514ac1e124d']
y = column_or_1d(y, warn=True)
indices = np.isin(y, _encoder.classes_)
y_transformed = np.searchsorted(_encoder.classes_, y)
y_transformed

In [ ]:
_encoder.classes_[3450]

In [ ]:
_encoder.classes_ = np.append(_encoder.classes_,['zzzz_000b4e02-7f43-11ec-ae40-78d9ac1e124d'])

In [ ]:
_encoder.transform(['ZZZZfgsf4'])

In [ ]:
sorted = np.sort(_encoder.classes_)
len(sorted)
np.where(sorted == 'zzzz_000b4e02-7f43-11ec-ae40-78d9ac1e124d')

In [ ]:
_encoder.transform(['zzzz_000b4e02-7f43-11ec-ae40-78d9ac1e124d'])

In [ ]:
sorted[3453]
type(sorted)

In [ ]:
check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        indices = np.isin(y, self.classes_)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.searchsorted(self.classes_, y)
        y_transformed[~indices]=self.unknown_encoded_value

In [ ]:
_encoder.transform(['PEPPOL'])
_encoder.transform(['SFID Metadata-Based Routing'])
_encoder.transform(['VAT.UK'])

In [ ]:
values = df.select([column]).drop_duplicates().toPandas()

In [ ]:
list(values['CSENDERENDPOINTID'])
values

## fix

In [ ]:
index = np.where(_encoder.classes_ == None)[0][0]
index

In [ ]:
_encoder.classes_ = np.delete( _encoder.classes_,index)

In [ ]:
_encoder.classes_

In [ ]:
np.save(new_directory + column + '.npy', _encoder.classes_)

In [ ]:
encoders[column] = _encoder

In [ ]:
#_encoder.transform(['SLMP_PLUS_MS'])
_encoder.transform([classes[9]])

## check original file

In [ ]:
#column = 'CSENDERPROTOCOL'
npy= share_directory + 'sla/' + version_sla + '/npy'
_encoder = TolerantLabelEncoder(ignore_unknown=True)
_encoder.classes_ = np.load(npy + '/' + column + '.npy')
_encoder.classes_

In [ ]:
index = np.where(_encoder.classes_ == None)[0][0]
index

In [ ]:
len(_encoder.classes_)

In [ ]:
_encoder.inverse_transform([5634])

In [ ]:
_encoder.transform(['fff8bb00-a213-11e9-a189-ddafac1b495c'])
#_encoder.transform(['VAT.UK'])

In [ ]:
#np.sort(_encoder.classes_)
#_encoder.transform(_encoder.classes_)
_encoder.classes_

In [ ]:
np.sort(['AS2', 'AS4', 'HTTP', 'MAIL', 'N/A', 'OFTP2', 'PEPPOL', 'QUEUE',
       'REST', 'SFTP', 'SLMP', 'SLMPPOLL', 'SLMP_PLUS_MS', 'unknown'])